In [ ]:
!pip install langchain-teddynote

In [ ]:
from google.colab import userdata
import os
os.environ["OPENAI_API_KEY"] = userdata.get("OPENAI_API_KEY")
os.environ["LANGSMITH_API_KEY"] = userdata.get("LANGSMITH_API_KEY")
os.environ["GEMINI_API_KEY"] = userdata.get("GEMINI_API_KEY")

from langchain_teddynote import logging
from langchain_teddynote.messages import stream_response

# 프로젝트 이름을 입력합니다.
logging.langsmith("CH04-Models")

In [ ]:
from langchain_openai import ChatOpenAI

# ChatOpenAI 객체를 생성합니다.
gpt = ChatOpenAI(
    temperature=0,
    model_name="gpt-4o",  # 모델명
)

# 스트리밍 출력을 위하여 invoke() 대신 stream()을 사용합니다.
answer = gpt.stream("사랑이 뭔가요?")

# 답변 출력
stream_response(answer)

In [ ]:
from langchain_teddynote.models import ChatPerplexity
os.environ["PPLX_API_KEY"] = userdata.get("PPLX_API_KEY")

perplexity = ChatPerplexity(
    model="llama-3-sonar-large-32k-online",
    temperature=0.2,
    top_p=0.9,
    search_domain_filter=["perplexity.ai"],
    return_images=False,
    return_related_questions=True,
    # search_recency_filter="month",
    top_k=0,
    streaming=False,
    presence_penalty=0,
    frequency_penalty=1,
)

# 응답 출력
response = perplexity.invoke("사랑이 뭔가요?")
print(response.content)

print()
for i, citation in enumerate(response.citations):
    print(f"[{i+1}] {citation}")


## Cache

In [ ]:
from langchain_openai import ChatOpenAI
from langchain_core.prompts import PromptTemplate

# 모델을 생성합니다.
llm = ChatOpenAI(model_name="gpt-3.5-turbo")

# 프롬프트를 생성합니다.
prompt = PromptTemplate.from_template("{country} 에 대해서 200자 내외로 요약해줘")

# 체인을 생성합니다.
chain = prompt | llm


In [ ]:
%%time
response = chain.invoke({"country": "한국"})
print(response.content)

### InMemoryCache

In [ ]:
!pip install -U langchain-community

In [ ]:
%%time
from langchain.globals import set_llm_cache
from langchain.cache import InMemoryCache

# 인메모리 캐시를 사용합니다.
set_llm_cache(InMemoryCache())

# 체인을 실행합니다.
response = chain.invoke({"country": "한국"})
print(response.content)

In [ ]:
%%time
# 체인을 실행합니다.
response = chain.invoke({"country": "한국"})
print(response.content)

###SQLite Cache

In [ ]:
from langchain_community.cache import SQLiteCache
from langchain_core.globals import set_llm_cache
import os

# 캐시 디렉토리를 생성합니다.
if not os.path.exists("cache"):
    os.makedirs("cache")

# SQLiteCache를 사용합니다.
set_llm_cache(SQLiteCache(database_path="cache/llm_cache.db"))

In [ ]:
%%time
# 체인을 실행합니다.
response = chain.invoke({"country": "한국"})
print(response.content)


In [ ]:
%%time
# 체인을 실행합니다.
response = chain.invoke({"country": "한국"})
print(response.content)


CPU time은 줄었지만 Wall time은 오히려 늘어난 현상이 나타났습니다. 이는 캐싱(Caching) 효과가 나타났음에도 불구하고, 다른 외부 요인 때문에 **전체 시간(Wall time)**이 더 오래 걸렸음을 의미

## 3. 모델 직렬화(Serialization) - 저장 및 불러오기

is_lc_serializable 클래스 메서드로 실행하여 LangChain 클래스가 직렬화 가능한지 확인할 수 있습니다.

## 5. 구글 생성 AI

In [ ]:
!pip install -qU langchain-google-genai

In [ ]:
from langchain_google_genai import ChatGoogleGenerativeAI

# ChatGoogleGenerativeAI 언어 모델을 초기화합니다.
llm = ChatGoogleGenerativeAI(model="gemini-1.5-pro-latest")

# 프롬프트를 전달하여 결과를 생성합니다.
answer = llm.stream("자연어처리에 대해서 간략히 설명해 줘")

# 결과를 출력합니다.
stream_response(answer)


# 6. 허깅페이스

In [ ]:
!pip install -qU huggingface_hub

In [ ]:
import os
from google.colab import userdata

# 코랩 Secret에 저장된 키를 환경 변수에 설정
os.environ["HF_TOKEN"] = userdata.get("HF_TOKEN")
os.environ["OPENAI_API_KEY"] = userdata.get("OPENAI_API_KEY")
os.environ["LANGSMITH_API_KEY"] = userdata.get("LANGSMITH_API_KEY")

In [ ]:
from huggingface_hub import login

login()

In [ ]:
from langchain.prompts import PromptTemplate

template = """<|system|>
You are a helpful assistant.<|end|>
<|user|>
{question}<|end|>
<|assistant|>"""

prompt = PromptTemplate.from_template(template)

### Serverless Endpoints

In [ ]:
!pip install -qU langchain-huggingface

설치하신 패키지들은 각각 다른 역할을 합니다.

huggingface_hub: Hugging Face Hub에 접근하고 모델, 데이터셋 등을 다운로드하는 데 필요한 라이브러리입니다.
langchain-teddynote: LangChain과 관련된 특정 기능이나 예제를 제공하는 라이브러리입니다.
langchain-huggingface: LangChain 프레임워크에서 Hugging Face 모델을 직접적으로 사용할 수 있도록 연결해주는 역할을 합니다.
따라서 langchain-huggingface 패키지를 설치해야 LangChain 코드에서 HuggingFaceEndpoint와 같은 클래스를 사용하여 Hugging Face 모델을 불러오고 사용할 수 있습니다.

repo_id 변수에 HuggingFace 모델의 repo ID(저장소 ID) 를 할당

In [ ]:
import os
from langchain_core.output_parsers import StrOutputParser
from langchain_huggingface import HuggingFaceEndpoint

# 사용할 모델의 저장소 ID를 설정합니다.
repo_id = "microsoft/Phi-3-mini-4k-instruct"

llm = HuggingFaceEndpoint(
    repo_id=repo_id,  # 모델 저장소 ID를 지정합니다.
    max_new_tokens=256,  # 생성할 최대 토큰 길이를 설정합니다.
    temperature=0.1,
    huggingfacehub_api_token=os.environ["HUGGINGFACEHUB_API_TOKEN"],  # 허깅페이스 토큰
)

# LLMChain을 초기화하고 프롬프트와 언어 모델을 전달합니다.
chain = prompt | llm | StrOutputParser()
# 질문을 전달하여 LLMChain을 실행하고 결과를 출력합니다.
response = chain.invoke({"question": "what is the capital of South Korea?"})
print(response)


In [ ]:
!pip install -U langchain-core langchain langchain-community langchain-huggingface langchain-openai langchain-google-genai

In [ ]:
import os
from langchain_core.output_parsers import StrOutputParser
from langchain_huggingface import HuggingFaceEndpoint
from langchain.prompts import PromptTemplate

# Re-define the prompt template if it was cleared
template = """<|system|>
You are a helpful assistant.<|end|>
<|user|>
{question}<|end|>
<|assistant|>"""

prompt = PromptTemplate.from_template(template)


# 사용할 모델의 저장소 ID를 설정합니다.
repo_id = "microsoft/Phi-3-mini-4k-instruct"

# 환경 변수에서 허깅페이스 토큰을 가져옵니다.
# Colab Secrets에 HUGGINGFACEHUB_API_TOKEN으로 저장되어 있어야 합니다.
hf_token = os.environ.get("HUGGINGFACEHUB_API_TOKEN")

# 토큰이 없으면 오류 메시지를 출력합니다.
if not hf_token:
    raise ValueError("HUGGINGFACEHUB_API_TOKEN not found in environment variables.")

llm = HuggingFaceEndpoint(
    repo_id=repo_id,  # 모델 저장소 ID를 지정합니다.
    max_new_tokens=256,  # 생성할 최대 토큰 길이를 설정합니다.
    temperature=0.1,
    huggingfacehub_api_token=hf_token,  # 허깅페이스 토큰
)

# LLMChain을 초기화하고 프롬프트와 언어 모델을 전달합니다.
chain = prompt | llm | StrOutputParser()

# 질문을 전달하여 LLMChain을 실행하고 결과를 출력합니다.
response = chain.invoke({"question": "what is the capital of South Korea?"})
print(response)

## 7. 허깅페이스 로컬(HuggingFace Local)

로컬 환경에서 허깅페이스 모델(Phi-3-mini-4k-instruct)을 불러와 랭체인(HuggingFacePipeline)을 통해 텍스트 생성 추론을 수행하는 과정 -> 아래 코드들

## 9. Ollama

Modelfile 로부터 커스텀 모델 생성하기

모델을 임포트하기 위해 ModelFile을 먼저 생성해야 합니다.